In [1]:
import numpy as np
from itertools import product
import pandas as pd
import os
import foraging_toolkit as ft
import environments
import simulation
import success_metrics
# import importlib
# importlib.reload(ft)
# importlib.reload(simulation)
# importlib.reload(environments)
# importlib.reload(success_metrics)

In [2]:
# choose a location to save the results data
home_dir = "/Users/emily/code/collaborative-intelligence/tmp_files/" 

Parameters of the forward model. Specify the parameter values for all the cases you are interested in. 

In [3]:
# agent parameters
sight_radius = [5]
c_trust = [0, 0.5] # 0: ignorers, 0.5: communicators (equal weight to own and others' knowledge of food)
N_agents = 9

# environment parameters
edge_size = 30
N_total_food_units = 16
reward_patch_dim = [1, 4] # clustered is 4, distributed is 1

# simulation parameters
N_runs = 20 # How many times would you like to run each case?
N_frames = 50

# Generate a dataframe containing all possible combinations of the parameter values specified above. 
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]
metadataDF = pd.DataFrame(param_list)
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"] 
metadataDF['sim index'] = np.arange(len(metadataDF)).astype(int)
N_sims = len(metadataDF)

# save metadata to home directory
metadataDF.to_csv(os.path.join(home_dir,'metadataDF.csv'))
pd.DataFrame([{'N_sims':N_sims, 'N_runs':N_runs, 'N_frames':N_frames, 'N_agents':N_agents, \
              'N_total_food_units':N_total_food_units, 'edge_size':edge_size}]).to_csv(os.path.join(home_dir, 'additional_meta_params.csv'))

display(metadataDF)

,c_trust,sight_radius,reward_patch_dim,sim index
0,0.0,5,1,0
1,0.0,5,4,1
2,0.5,5,1,2
3,0.5,5,4,3


In [4]:
# run simulations

# data frame to store all success values across all simulations and runs of each simulation
all_results = []

for si in range(N_sims):
    
    # 1. pull out parameters from row si in the metadata
    df_row = metadataDF.iloc[[si]]
    c_trust = df_row['c_trust'].iloc[0]
    sight_radius = df_row['sight_radius'].iloc[0]
    reward_patch_dim = df_row['reward_patch_dim'].iloc[0].astype(int)

    # arrays to save success measures for each run of this simulation
    mean_times_to_first_reward = np.zeros((N_runs))
    num_birds_failed = np.zeros((N_runs))

    # Do multiple runs of the simulation and store the results in a results dataframe
    for ri in range(N_runs):
        # initialize environment
        env = environments.Environment(edge_size=edge_size, N_total_food_units=N_total_food_units, patch_dim=reward_patch_dim)
        env.add_food_patches()

        # run simulation
        sim = simulation.SimulateCommunicators(env, N_frames, N_agents, c_trust=c_trust, sight_radius=sight_radius)
        sim.run()
        
        # Compute success measures 
        time_to_first_allbirds = np.zeros(N_agents)
        for bird_id in range(1, N_agents + 1): # compute time to first food for each bird 
            singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF.bird == bird_id]  
            time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)
        mean_times_to_first_reward = np.mean(time_to_first_allbirds) # take the average across birds
        num_birds_failed = np.sum(time_to_first_allbirds == N_frames) # number of birds that failed to reach food

        # Save the simulation results in a folder named sim{si}_run{ri} in the home directory
        sim_folder = 'sim' + str(si) + '_run' + str(ri)
        sim_dir = os.path.join(home_dir, sim_folder)
        if not os.path.isdir(sim_dir):
            os.makedirs(sim_dir)
        sim.all_birdsDF.to_csv(os.path.join(sim_dir, "birdlocsDF.csv"))
        sim.all_rewardsDF.to_csv(os.path.join(sim_dir, "rewardlocsDF.csv"))

        #Combine the metadata and the success measures for the results dataframe 
        results_onesim = {
               "c_trust": c_trust,
                "sight_radius": sight_radius,
                "reward_patch_dim": reward_patch_dim,
                "sim index": si,
                "run index": ri,
                "time to first food": mean_times_to_first_reward,
                "num birds failed": num_birds_failed,
            }
        all_results.append(results_onesim)


# save metadata to home directory
allresultsDF = pd.DataFrame(all_results)
allresultsDF.to_csv(os.path.join(home_dir, 'resultsDF.csv'))
display(allresultsDF)

In [ ]:
# animate a simulation
# load the data from the one simulation
sim_folder = 'sim0_run0'
sim_dir = os.path.join(home_dir, sim_folder)
birdlocsDF = pd.read_csv(os.path.join(sim_dir, "birdlocsDF.csv"), index_col=0)
rewardlocsDF = pd.read_csv(os.path.join(sim_dir, "rewardlocsDF.csv"), index_col=0)

# animate the last simulation
communicators = ft.object_from_data(birdlocsDF, rewardlocsDF)
ft.animate_birds(communicators, plot_rewards=True,
                  width = 600, height = 600, point_size = 10)

See histograms.ipynb to generate histograms